# Data restructurisation

We will work on dataset from JustJoinIT website, which is a job portal for IT specialists.

The range of our data set is from 10.2021 to 09.2023

In [1]:
import os
import ijson
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/tymoteuszhuba/Desktop/Python/my_projects/JustJoinIT_project/processed_data.csv')
df.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,skills,employment_types,published_at
0,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",2022-06-07T12:00:00.000Z
1,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",2022-05-07T12:00:00.000Z
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","[{'type': 'permanent', 'salary': {'from': 1500...",2022-04-11T17:33:00.000Z
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",2022-04-11T17:32:00.000Z
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",2022-04-11T17:03:00.000Z


First we need to extract elements from skills. As we can see skills column have a list elements, we need to change the format to show better list elements separate by comma.

Before we extract data, we need to change the format for skills and employment_types by using a parse_values function

In [3]:
# Konwersja ciągu tekstowego na listę słowników
def parse_values(val):
    if isinstance(val, str):  # Jeśli dane to ciąg znaków
        try:
            return json.loads(val.replace("'", '"'))  # Zamiana ' na " dla poprawnego JSON
        except json.JSONDecodeError:
            return None
    return val  # Jeśli dane już są listą, zwróć bez zmian

In [4]:
df['skills'] = df['skills'].apply(parse_values)

Now we have corectly prepared skills for extracting. For that we use extract_skills function like below

In [5]:
# Następnie użyj extract_skills
def extract_skills(skills):
    if isinstance(skills, list):
        return ', '.join([f"{skill.get('name', 'Unknown')} (level {skill.get('level', 'N/A')})" for skill in skills])
    return None

df['skills_extracted'] = df['skills'].apply(extract_skills)

df[['skills', 'skills_extracted']].head()

,skills,skills_extracted
0,"[{'name': 'С++', 'level': 4}]",С++ (level 4)
1,"[{'name': 'С++', 'level': 4}]",С++ (level 4)
2,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","GCloud (level 3), JavaScript (level 3), Django..."
3,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","Git (level 3), Java (level 4), Android (level 4)"
4,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","Agile (level 2), JIRA (level 2), English (leve..."


The same we need to do with our employment_types.

In [6]:
# Funkcja przetwarzająca employment_types
def parse_employment_types(employment_types):
    if isinstance(employment_types, str):  # Jeśli dane są ciągiem znaków
        try:
            # Zamień 'None' na 'null' oraz ' na ", aby dane były poprawnym JSON
            formatted_string = employment_types.replace("'", '"').replace("None", "null")
            return json.loads(formatted_string)  # Konwersja na strukturę danych Python
        except json.JSONDecodeError as e:
            print(f"Błąd parsowania JSON: {e}, dane: {employment_types}")
            return None
    elif isinstance(employment_types, list):  # Jeśli dane są już listą
        return employment_types
    return None  # W innych przypadkach zwróć None

In [7]:
df['employment_types_parsed'] = df['employment_types'].apply(parse_employment_types)

# Podgląd wyników
print(df[['employment_types', 'employment_types_parsed']].head())


                                    employment_types  \
0            [{'type': 'permanent', 'salary': None}]   
1            [{'type': 'permanent', 'salary': None}]   
2  [{'type': 'permanent', 'salary': {'from': 1500...   
3  [{'type': 'b2b', 'salary': {'from': 15000, 'to...   
4  [{'type': 'b2b', 'salary': {'from': 5400, 'to'...   

                             employment_types_parsed  
0            [{'type': 'permanent', 'salary': None}]  
1            [{'type': 'permanent', 'salary': None}]  
2  [{'type': 'permanent', 'salary': {'from': 1500...  
3  [{'type': 'b2b', 'salary': {'from': 15000, 'to...  
4  [{'type': 'b2b', 'salary': {'from': 5400, 'to'...  


In [8]:
# Funkcja rozwijająca employment_types na czytelne informacje
def expand_employment_details(employment_types):
    if isinstance(employment_types, list):  # Sprawdź, czy dane to lista
        results = []
        for emp in employment_types:
            emp_type = emp.get('type', 'Unknown')
            salary = emp.get('salary', {})
            if salary:
                salary_from = salary.get('from', 'N/A')
                salary_to = salary.get('to', 'N/A')
                currency = salary.get('currency', 'N/A')
                results.append(f"{emp_type}: {salary_from}-{salary_to} {currency}")
            else:
                results.append(f"{emp_type}: No salary info")
        return "; ".join(results)
    return None

# Przekształcenie danych
df['employment_details'] = df['employment_types_parsed'].apply(expand_employment_details)

# Podgląd wyników
df[['employment_types', 'employment_details']].head()


,employment_types,employment_details
0,"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
1,"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
2,"[{'type': 'permanent', 'salary': {'from': 1500...",permanent: 15000-25000 pln
3,"[{'type': 'b2b', 'salary': {'from': 15000, 'to...",b2b: 15000-25000 pln
4,"[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",b2b: 5400-7300 pln; permanent: 4500-6100 pln


In [9]:
df.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,skills,employment_types,published_at,skills_extracted,employment_types_parsed,employment_details
0,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",2022-06-07T12:00:00.000Z,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
1,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",2022-05-07T12:00:00.000Z,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","[{'type': 'permanent', 'salary': {'from': 1500...",2022-04-11T17:33:00.000Z,"GCloud (level 3), JavaScript (level 3), Django...","[{'type': 'permanent', 'salary': {'from': 1500...",permanent: 15000-25000 pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",2022-04-11T17:32:00.000Z,"Git (level 3), Java (level 4), Android (level 4)","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",b2b: 15000-25000 pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",2022-04-11T17:03:00.000Z,"Agile (level 2), JIRA (level 2), English (leve...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",b2b: 5400-7300 pln; permanent: 4500-6100 pln


In [10]:
df.shape

(8852237, 15)

### Time format

Let's change the format of the columm published_at

In [11]:
df['published_at'].astype

<bound method NDFrame.astype of 0          2022-06-07T12:00:00.000Z
1          2022-05-07T12:00:00.000Z
2          2022-04-11T17:33:00.000Z
3          2022-04-11T17:32:00.000Z
4          2022-04-11T17:03:00.000Z
                     ...           
8852232    2023-09-14T09:22:00.000Z
8852233    2023-09-14T09:21:00.000Z
8852234    2023-09-13T13:00:12.018Z
8852235    2023-09-12T13:13:00.000Z
8852236    2023-09-12T13:13:00.000Z
Name: published_at, Length: 8852237, dtype: object>

In [12]:
df['published_at'] = pd.to_datetime(df['published_at']).dt.strftime('%d-%m-%y')

df.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,skills,employment_types,published_at,skills_extracted,employment_types_parsed,employment_details
0,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-06-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
1,SENIOR UI DEVELOPER,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-05-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","[{'type': 'permanent', 'salary': {'from': 1500...",11-04-22,"GCloud (level 3), JavaScript (level 3), Django...","[{'type': 'permanent', 'salary': {'from': 1500...",permanent: 15000-25000 pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",11-04-22,"Git (level 3), Java (level 4), Android (level 4)","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",b2b: 15000-25000 pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",11-04-22,"Agile (level 2), JIRA (level 2), English (leve...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",b2b: 5400-7300 pln; permanent: 4500-6100 pln


### Change text size 

We will normalize the letters size in the tittle column

In [13]:
df['title'].dtype

dtype('O')

In [14]:
df['title'] = df['title'].str.title()
df.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,skills,employment_types,published_at,skills_extracted,employment_types_parsed,employment_details
0,Senior Ui Developer,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-06-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
1,Senior Ui Developer,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-05-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent: No salary info
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","[{'type': 'permanent', 'salary': {'from': 1500...",11-04-22,"GCloud (level 3), JavaScript (level 3), Django...","[{'type': 'permanent', 'salary': {'from': 1500...",permanent: 15000-25000 pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",11-04-22,"Git (level 3), Java (level 4), Android (level 4)","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",b2b: 15000-25000 pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",11-04-22,"Agile (level 2), JIRA (level 2), English (leve...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",b2b: 5400-7300 pln; permanent: 4500-6100 pln


### Extract data

We will extract data from employment_details for employment, salary from to and currency

In [15]:
# Funkcja do przekształcania employment_details na osobne wiersze
def expand_employment_rows(row):
    employment_details = row['employment_details']
    if isinstance(employment_details, str):
        entries = employment_details.split("; ")
        expanded_rows = []
        for entry in entries:
            emp_type, *salary_info = entry.split(": ")
            salary_info = salary_info[0] if salary_info else "No salary info"
            salary_from, salary_to, currency = ("No info", "No info", "N/A")
            if "-" in salary_info:
                try:
                    salary_range, currency = salary_info.rsplit(" ", 1)
                    salary_from, salary_to = salary_range.split("-")
                except ValueError:
                    pass
            expanded_rows.append({
                **row.to_dict(),
                "employment": emp_type.strip(),
                "salary_from": salary_from.strip(),
                "salary_to": salary_to.strip(),
                "currency": currency.strip()
            })
        return expanded_rows
    return [row.to_dict()]  # Wiersz pozostaje niezmieniony, jeśli brak szczegółów


In [16]:
# Przekształcanie DataFrame na podstawie funkcji expand_employment_rows
expanded_data = []

for _, row in df.iterrows():
    expanded_data.extend(expand_employment_rows(row))

# Stworzenie nowego DataFrame
df_expanded = pd.DataFrame(expanded_data)

# Usuwanie starych kolumn (opcjonalnie)
df_expanded.drop(columns=['employment_details'], inplace=True)

# Podgląd wyników
df_expanded.head()


,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,skills,employment_types,published_at,skills_extracted,employment_types_parsed,employment,salary_from,salary_to,currency
0,Senior Ui Developer,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-06-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent,No info,No info,N/A
1,Senior Ui Developer,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,"[{'name': 'С++', 'level': 4}]","[{'type': 'permanent', 'salary': None}]",07-05-22,С++ (level 4),"[{'type': 'permanent', 'salary': None}]",permanent,No info,No info,N/A
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'GCloud', 'level': 3}, {'name': 'Jav...","[{'type': 'permanent', 'salary': {'from': 1500...",11-04-22,"GCloud (level 3), JavaScript (level 3), Django...","[{'type': 'permanent', 'salary': {'from': 1500...",permanent,15000,25000,pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,"[{'name': 'Git', 'level': 3}, {'name': 'Java',...","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",11-04-22,"Git (level 3), Java (level 4), Android (level 4)","[{'type': 'b2b', 'salary': {'from': 15000, 'to...",b2b,15000,25000,pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,"[{'name': 'Agile', 'level': 2}, {'name': 'JIRA...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",11-04-22,"Agile (level 2), JIRA (level 2), English (leve...","[{'type': 'b2b', 'salary': {'from': 5400, 'to'...",b2b,5400,7300,pln


In [18]:
df_expanded.shape

(11367731, 18)

### Cleaning df_expanded data

In [20]:
df_expanded.drop(columns = ['skills', 'employment_types', 'employment_types_parsed'], inplace=True)
df_expanded.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,published_at,skills_extracted,employment,salary_from,salary_to,currency
0,Senior Ui Developer,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,07-06-22,С++ (level 4),permanent,No info,No info,N/A
1,Senior Ui Developer,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,07-05-22,С++ (level 4),permanent,No info,No info,N/A
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,11-04-22,"GCloud (level 3), JavaScript (level 3), Django...",permanent,15000,25000,pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,11-04-22,"Git (level 3), Java (level 4), Android (level 4)",b2b,15000,25000,pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,11-04-22,"Agile (level 2), JIRA (level 2), English (leve...",b2b,5400,7300,pln


### Rename columns

In [22]:
df_expanded = df_expanded.rename(columns = {'skills_extracted' : 'skills'})
df_expanded.head()

,title,marker_icon,workplace_type,company_name,city,country_code,company_size,experience_level,remote,published_at,skills,employment,salary_from,salary_to,currency
0,Senior Ui Developer,game,remote,Dragons Lake,Wrocław,PL,170+,senior,True,07-06-22,С++ (level 4),permanent,No info,No info,N/A
1,Senior Ui Developer,game,remote,Dragons Lake,Poznań,PL,170+,senior,True,07-05-22,С++ (level 4),permanent,No info,No info,N/A
2,Django Full Stack Developer,python,remote,MIM Solutions,Warszawa,PL,30+,senior,True,11-04-22,"GCloud (level 3), JavaScript (level 3), Django...",permanent,15000,25000,pln
3,Android Developer,mobile,remote,MIM Solutions,Warszawa,PL,30+,senior,True,11-04-22,"Git (level 3), Java (level 4), Android (level 4)",b2b,15000,25000,pln
4,Product Specialist - Technical Support,support,remote,Kitopi,Kraków,PL,90,junior,True,11-04-22,"Agile (level 2), JIRA (level 2), English (leve...",b2b,5400,7300,pln


### Save into csv file

Prepared data we will save into a prepared_data.csv

In [23]:
df_expanded.to_csv('/Users/tymoteuszhuba/Desktop/Python/my_projects/JustJoinIT_project/prepared_data.csv', index=False)